In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences. # the second col is just an indicator for the book that it is included in
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words, minus stops and punctuation
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lhamlet_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    # NOTE: the columns 0 and 1 referenced below are not the 1st and second sentences, they are
    # REFERENCES to the 0 and 1 columns in the DF with the sentences
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    # Setting these all equal to zero
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lhamlets, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lhamlet_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lhamlet_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
hamlet_bow = bow_features(sentences, common_words)
hamlet_bow.head()

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing row 4900
Processing r

,stay,reflection,wretched,mock,shew,prospect,welcome,leaf,street,recover,...,golden,comfort,indulgence,the,number,bread,pack,opinion,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


Now time to model...

__Random Forest Classifier__

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = hamlet_bow['text_source']
X = np.array(hamlet_bow.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.987460815047

Test set score: 0.888157894737


__Logistic Regression__

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 1561) (3190,)
Training set score: 0.952351097179

Test set score: 0.917763157895


__Gradient Boosted Classifier__

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.886833855799

Test set score: 0.874060150376


Now let's try a different novel by Jane Austen, *hamlet*, and see if it can be distinguished.

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.693512304251


col_0,Austen,Carroll
row_0,,
Austen,1558,111
Carroll,711,302


Not bad, so let's do a couple challenges...

# Challenge 1:
Improve the model's performance on the test set. Let's mess with SVM for starters.

In [24]:
from sklearn import svm
svc = svm.SVC()

train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(3190, 1561) (3190,)
Training set score: 0.685893416928

Test set score: 0.692669172932


Doesn't look like SVM is likely to improve our results much, so let's instead optimize our logreg through gridsearch

In [33]:
from sklearn.model_selection import GridSearchCV

parameters = {'penalty':['l1','l2'], 'C':[0.9,0.95,1,2,3,4,5,10,100,1000,10000]}

clf = GridSearchCV(lr,parameters)
clf.fit(X,Y)
best_params = clf.best_params_

In [34]:
best_params

{'C': 0.95, 'penalty': 'l2'}

In [35]:
lr = LogisticRegression(**best_params)
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 1561) (3190,)
Training set score: 0.951410658307

Test set score: 0.918233082707


This gets us a very modest improvement, which is something!

# Challenge 2
See if our new model is any good at predicting a new novel vs. Alice in Wonderland or Austen vs. another work.

Let's pick something else from Gutenberg.

In [46]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [47]:
# Clean the hamlet data.
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
hamlet = re.sub(r'VOLUME \w+', '', hamlet)
hamlet = re.sub(r'CHAPTER \w+', '', hamlet)
hamlet = text_cleaner(hamlet)
print(hamlet[:100])

Actus Primus. Scoena Prima. Enter Barnardo and Francisco two Centinels. Barnardo. Who's there? Fran.


In [48]:
# Parse our cleaned data.
hamlet_doc = nlp(hamlet)

In [53]:
# Group into sentences.
persuasion_sents = [[sent, "Shakespeare"] for sent in persuasion_doc.sents]
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]

# hamlet is also long, so let's cut sentences down to length of alice
hamlet_sents = hamlet_sents[0:len(alice_sents)]

In [54]:
# Build a new Bag of Words data frame for hamlet word counts.
# We'll use the same common words from Alice and Persuasion.
hamlet_sentences = pd.DataFrame(hamlet_sents)
hamlet_bow = bow_features(hamlet_sentences, common_words)

print('done')

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
done


In [55]:
# Modeling again using logistic regression
X_hamlet_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    hamlet_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_hamlet_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Shakespeare'] * hamlet_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_hamlet_test, y_hamlet_test))
lr_hamlet_predicted = lr.predict(X_hamlet_test)
pd.crosstab(y_hamlet_test, lr_hamlet_predicted)


Test set score: 0.112229679344


col_0,Austen,Carroll
row_0,,
Carroll,712,301
Shakespeare,1390,279


The model didn't do a great job with this new data, mischaracterizing ~70% (712/1013) instances of the Carroll novel, but this is a bit unfair. Let's see how it does when refit to actual Shakespeare data.

In [77]:
carroll = word_counts[word_counts['text_source'] == 'Carroll']

In [85]:
X = np.concatenate((carroll.drop(['text_sentence','text_source'], 1),
    hamlet_bow.drop(['text_sentence','text_source'], 1)))
Y = np.concatenate((carroll['text_source'],
    hamlet_bow['text_source']),axis=0)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)
lr = LogisticRegression(**best_params)
train = lr.fit(X_train, y_train)

In [92]:
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2002, 1561) (2002,)
Training set score: 0.929070929071

Test set score: 0.86377245509


The model does pretty good once it's refit to the new data, even if the original model doesn't do so well when just trying to identify Carroll.